In [1]:
import numpy as np
import pandas as pd
import fastai
from tqdm import tqdm_notebook as tqdm
from fastai.tabular import *
import pickle

from multiprocessing import Pool
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, StandardScaler
np.range = (lambda x:(x.min(), x.max()))

In [2]:
DATA_PATH = '../input'
SUBMISSIONS_PATH = './'
# use atomic numbers to recode atomic names
ATOMIC_NUMBERS = {
    'H': 1,
    'C': 6,
    'N': 7,
    'O': 8,
    'F': 9
}

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)

In [4]:
train_dtypes = {
    'molecule_name': 'category',
    'atom_index_0': 'int8',
    'atom_index_1': 'int8',
    'type': 'category',
    'scalar_coupling_constant': 'float32'
}
train_csv = pd.read_csv(f'{DATA_PATH}/train.csv', index_col='id', dtype=train_dtypes)
#train_csv['molecule_name'] = train_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
train_csv = train_csv[['molecule_name', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant']]
train_csv.head(10)

/opt/conda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
id,,,,,
0,dsgdb9nsd_000001,1,0,1JHC,84.807602
1,dsgdb9nsd_000001,1,2,2JHH,-11.257000
2,dsgdb9nsd_000001,1,3,2JHH,-11.254800
3,dsgdb9nsd_000001,1,4,2JHH,-11.254300
4,dsgdb9nsd_000001,2,0,1JHC,84.807404
5,dsgdb9nsd_000001,2,3,2JHH,-11.254100
6,dsgdb9nsd_000001,2,4,2JHH,-11.254800
7,dsgdb9nsd_000001,3,0,1JHC,84.809303
8,dsgdb9nsd_000001,3,4,2JHH,-11.254300


In [5]:
test_csv = pd.read_csv(f'{DATA_PATH}/test.csv', index_col='id', dtype=train_dtypes)
#test_csv['molecule_index'] = test_csv['molecule_name'].str.replace('dsgdb9nsd_', '').astype('int32')
test_csv = test_csv[['molecule_name', 'atom_index_0', 'atom_index_1', 'type']]
test_csv.head(10)

,molecule_name,atom_index_0,atom_index_1,type
id,,,,
4658147,dsgdb9nsd_000004,2,0,2JHC
4658148,dsgdb9nsd_000004,2,1,1JHC
4658149,dsgdb9nsd_000004,2,3,3JHH
4658150,dsgdb9nsd_000004,3,0,1JHC
4658151,dsgdb9nsd_000004,3,1,2JHC
4658152,dsgdb9nsd_000015,3,0,1JHC
4658153,dsgdb9nsd_000015,3,2,3JHC
4658154,dsgdb9nsd_000015,3,4,2JHH
4658155,dsgdb9nsd_000015,3,5,2JHH


In [6]:
structures_dtypes = {
    'molecule_name': 'category',
    'atom_index': 'int8',
    'atom': 'category',
    'x': 'float32',
    'y': 'float32',
    'z': 'float32'
}
structures_csv = pd.read_csv(f'{DATA_PATH}/structures.csv', dtype=structures_dtypes)
#structures_csv['molecule_index'] = structures_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
structures_csv = structures_csv[['molecule_name', 'atom_index', 'atom', 'x', 'y', 'z']]
structures_csv['atom'] = structures_csv['atom'].replace(ATOMIC_NUMBERS).astype('int8')
structures_csv.head(10)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,6,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,1,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,1,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,1,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,1,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,0,7,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,1,1,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,2,1,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,3,1,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,0,8,-0.034360,0.977540,0.007602


In [7]:
def build_type_dataframes(base, structures, coupling_type):
    base = base[base['type'] == coupling_type].drop('type', axis=1).copy()
    base = base.reset_index()
    base['id'] = base['id'].astype('int32')
    structures = structures[structures['molecule_name'].isin(base['molecule_name'])]
    return base, structures


def add_coordinates(base, structures, index):
    df = pd.merge(base, structures, how='inner',
                  left_on=['molecule_name', f'atom_index_{index}'],
                  right_on=['molecule_name', 'atom_index']).drop(['atom_index'], axis=1)
    df = df.rename(columns={
        'atom': f'atom_{index}',
        'x': f'x_{index}',
        'y': f'y_{index}',
        'z': f'z_{index}'
    })
    return df

def add_atoms(base, atoms):
    df = pd.merge(base, atoms, how='inner',
                  on=['molecule_name', 'atom_index_0', 'atom_index_1'])
    return df


def merge_all_atoms(base, structures):
    df = pd.merge(base, structures, how='left',
                  left_on=['molecule_name'],
                  right_on=['molecule_name'])
    df = df[(df.atom_index_0 != df.atom_index) & (df.atom_index_1 != df.atom_index)]
    return df

def add_center(df):
    df['x_c'] = ((df['x_1'] + df['x_0']) * np.float32(0.5))
    df['y_c'] = ((df['y_1'] + df['y_0']) * np.float32(0.5))
    df['z_c'] = ((df['z_1'] + df['z_0']) * np.float32(0.5))

def add_distance_to_center(df):
    df['d_c'] = ((
        (df['x_c'] - df['x'])**np.float32(2) +
        (df['y_c'] - df['y'])**np.float32(2) + 
        (df['z_c'] - df['z'])**np.float32(2)
    )**np.float32(0.5))

def add_distance_between(df, suffix1, suffix2):
    df[f'd_{suffix1}_{suffix2}'] = ((
        (df[f'x_{suffix1}'] - df[f'x_{suffix2}'])**np.float32(2) +
        (df[f'y_{suffix1}'] - df[f'y_{suffix2}'])**np.float32(2) + 
        (df[f'z_{suffix1}'] - df[f'z_{suffix2}'])**np.float32(2)
    )**np.float32(0.5))
    
    
def add_distances(df):
    n_atoms = 1 + max([int(c.split('_')[1]) for c in df.columns if c.startswith('x_')])
    
    for i in range(1, n_atoms):
        for vi in range(min(4, i)):
            add_distance_between(df, i, vi)
            
def add_n_atoms(base, structures):
    dfs = structures['molecule_name'].value_counts().rename('n_atoms').to_frame()
    return pd.merge(base, dfs, left_on='molecule_name', right_index=True)

def build_couple_dataframe(some_csv, structures_csv, coupling_type, n_atoms=10):
    base, structures = build_type_dataframes(some_csv, structures_csv, coupling_type)
    base = add_coordinates(base, structures, 0)
    base = add_coordinates(base, structures, 1)
    
    base = base.drop(['atom_0', 'atom_1'], axis=1)
    atoms = base.drop('id', axis=1).copy()
    if 'scalar_coupling_constant' in some_csv:
        atoms = atoms.drop(['scalar_coupling_constant'], axis=1)
        
    add_center(atoms)
    atoms = atoms.drop(['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1'], axis=1)

    atoms = merge_all_atoms(atoms, structures)
    
    add_distance_to_center(atoms)
    
    atoms = atoms.drop(['x_c', 'y_c', 'z_c', 'atom_index'], axis=1)
    atoms.sort_values(['molecule_name', 'atom_index_0', 'atom_index_1', 'd_c'], inplace=True)
    atom_groups = atoms.groupby(['molecule_name', 'atom_index_0', 'atom_index_1'])
    atoms['num'] = atom_groups.cumcount() + 2
    atoms = atoms.drop(['d_c'], axis=1)
    atoms = atoms[atoms['num'] < n_atoms]

    atoms = atoms.set_index(['molecule_name', 'atom_index_0', 'atom_index_1', 'num']).unstack()
    atoms.columns = [f'{col[0]}_{col[1]}' for col in atoms.columns]
    atoms = atoms.reset_index()
    
    # downcast back to int8
    for col in atoms.columns:
        if col.startswith('atom_'):
            atoms[col] = atoms[col].fillna(0).astype('int8')
            
    #atoms['molecule_name'] = atoms['molecule_name'].astype('int32')
    
    full = add_atoms(base, atoms)
    add_distances(full)
    
    full.sort_values('id', inplace=True)
    
    return full


def angle(df, u=['x_0', 'y_0', 'z_0'], v=['x_1', 'y_1', 'z_1']):
    u, v = df[u].values, df[v].values
    c = (u*v).sum(-1)/np.linalg.norm(u, axis=-1)/np.linalg.norm(v, axis=-1)
    c = c.clip(-1, 1)
    return np.arccos(c)

In [8]:
train_pre, test_pre = None, None
for t in tqdm(['3JHC', '2JHC', '1JHC', '3JHH', '2JHH', '3JHN', '2JHN', '1JHN']):
    df = build_couple_dataframe(train_csv, structures_csv, t, n_atoms=7)
    df['type'] = t
    df = df.drop(columns=['x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 'z_2', 'z_3', 'z_4', 'z_5', 'z_6'])
    if train_pre is None: train_pre = df
    else: train_pre = train_pre.append(df)
        
    df = build_couple_dataframe(test_csv, structures_csv, t, n_atoms=7)
    df['type'] = t
    df = df.drop(columns=['x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 'z_2', 'z_3', 'z_4', 'z_5', 'z_6'])
    if test_pre is None: test_pre = df
    else: test_pre = test_pre.append(df)
        
test_pre = test_pre.fillna(0).set_index('id')
train_pre = train_pre.fillna(0).set_index('id')

train_pre['angle'] = angle(train_pre)
test_pre['angle']  = angle(test_pre)

#train_pre = train_pre.drop(columns=['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1'])
#test_pre  = test_pre.drop( columns=['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1'])

test_pre = test_pre.sort_index()
train_pre = train_pre.sort_index()
test_pre = test_pre.reset_index()
train_pre = train_pre.reset_index()

In [9]:
train_csv.shape, train_pre.shape, test_pre.shape, test_csv.shape

((4658147, 5), (4658147, 36), (2505542, 35), (2505542, 4))

In [10]:
train_pre = train_pre.set_index('id')
train_pre = train_pre.sort_index()
train_pre = train_pre.reset_index()

In [11]:
train_pre.head()

,id,molecule_name,atom_index_0,atom_index_1,scalar_coupling_constant,x_0,y_0,z_0,x_1,y_1,z_1,atom_2,atom_3,atom_4,atom_5,atom_6,d_1_0,d_2_0,d_2_1,d_3_0,d_3_1,d_3_2,d_4_0,d_4_1,d_4_2,d_4_3,d_5_0,d_5_1,d_5_2,d_5_3,d_6_0,d_6_1,d_6_2,d_6_3,type,angle
0,0,dsgdb9nsd_000001,1,0,84.807602,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1,1,1,0,0,1.091953,1.783120,1.091952,1.783147,1.091946,1.783157,1.783157,1.091948,1.783148,1.783148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1JHC,2.694086
1,1,dsgdb9nsd_000001,1,2,-11.257000,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,6,1,1,0,0,1.783120,1.091953,1.091952,1.783157,1.783148,1.091948,1.783147,1.783157,1.091946,1.783148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2JHH,2.162629
2,2,dsgdb9nsd_000001,1,3,-11.254800,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,6,1,1,0,0,1.783147,1.091953,1.091946,1.783120,1.783157,1.091952,1.783157,1.783148,1.091948,1.783148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2JHH,2.924457
3,3,dsgdb9nsd_000001,1,4,-11.254300,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,6,1,1,0,0,1.783157,1.091953,1.091948,1.783120,1.783148,1.091952,1.783147,1.783148,1.091946,1.783157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2JHH,2.307507
4,4,dsgdb9nsd_000001,2,0,84.807404,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1,1,1,0,0,1.091952,1.783120,1.091953,1.783148,1.091948,1.783157,1.783157,1.091946,1.783147,1.783148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1JHC,0.616519


In [12]:
train_csv.head()

,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
id,,,,,
0,dsgdb9nsd_000001,1,0,1JHC,84.807602
1,dsgdb9nsd_000001,1,2,2JHH,-11.257000
2,dsgdb9nsd_000001,1,3,2JHH,-11.254800
3,dsgdb9nsd_000001,1,4,2JHH,-11.254300
4,dsgdb9nsd_000001,2,0,1JHC,84.807404


In [13]:
structures_csv[structures_csv.molecule_name=='dsgdb9nsd_000001']

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,6,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,1,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,1,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,1,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,1,-0.523814,1.437933,0.906397


In [14]:
train_pre['type_i'] = train_pre.type.apply(lambda x: int(list(x)[0])-1).astype(np.int8)
train_pre['type_a'] = train_pre.type.apply(lambda x: list(x)[-1])

test_pre['type_i'] = test_pre.type.apply(lambda x: int(list(x)[0])-1).astype(np.int8)
test_pre['type_a'] = test_pre.type.apply(lambda x: list(x)[-1])

type_a_encoder   = LabelEncoder().fit(train_pre.type_a)
train_pre.type_a = type_a_encoder.transform(train_pre.type_a).astype(np.int8)
test_pre.type_a  = type_a_encoder.transform(test_pre.type_a).astype(np.int8)

type_encoder = LabelEncoder().fit(train_pre.type)
train_pre.type = type_encoder.transform(train_pre.type).astype(np.int8)
train_pre.atom_index_0 += 1
train_pre.atom_index_1 += 1

test_pre.type = type_encoder.transform(test_pre.type).astype(np.int8)
test_pre.atom_index_0 += 1
test_pre.atom_index_1 += 1

In [15]:
atom_encoder = LabelEncoder().fit([0,1,6,7,8,9])
for atom_name in ['atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6']:
    train_pre[atom_name] = atom_encoder.transform(train_pre[atom_name]).astype(np.int8)
    test_pre[atom_name ] = atom_encoder.transform(test_pre[atom_name ]).astype(np.int8)

In [16]:
len(train_pre.columns)

38

In [17]:
column_names = ['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type_i', 'type_a', 'type', 'atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6', 'angle', 
                'd_1_0', 'd_2_0', 'd_2_1', 'd_3_0', 'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1', 'd_4_2', 'd_4_3', 'd_5_0', 
                'd_5_1', 'd_5_2', 'd_5_3','d_6_0', 'd_6_1', 'd_6_2', 'd_6_3',
                'scalar_coupling_constant']
train_pre = train_pre[column_names]

test_pre =  test_pre[column_names[:-1]]

In [18]:
len(column_names)

32

In [19]:
train_pre.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type_i',
       'type_a', 'type', 'atom_2', 'atom_3', 'atom_4', 'atom_5', 'atom_6',
       'angle', 'd_1_0', 'd_2_0', 'd_2_1', 'd_3_0', 'd_3_1', 'd_3_2', 'd_4_0',
       'd_4_1', 'd_4_2', 'd_4_3', 'd_5_0', 'd_5_1', 'd_5_2', 'd_5_3', 'd_6_0',
       'd_6_1', 'd_6_2', 'd_6_3', 'scalar_coupling_constant'],
      dtype='object')

In [20]:
conts = ['angle', 'd_1_0', 'd_2_0', 'd_2_1', 'd_3_0', 'd_3_1', 'd_3_2', 'd_4_0', 'd_4_1', 'd_4_2', 'd_4_3', 
         'd_5_0', 'd_5_1', 'd_5_2', 'd_5_3','d_6_0', 'd_6_1', 'd_6_2', 'd_6_3',
        ]

### standardize

In [21]:
for cont in tqdm(conts):
    m, s = train_pre[cont].mean(), train_pre[cont].std()
    train_pre[cont] = ((train_pre[cont] - m)/s).astype(np.float32)
    test_pre[cont ] = ((test_pre[cont]  - m)/s).astype(np.float32)

In [22]:
train_pre[conts].describe()

,angle,d_1_0,d_2_0,d_2_1,d_3_0,d_3_1,d_3_2,d_4_0,d_4_1,d_4_2,d_4_3,d_5_0,d_5_1,d_5_2,d_5_3,d_6_0,d_6_1,d_6_2,d_6_3
count,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06
mean,-1.749185e-03,1.857403e-02,6.583207e-02,6.287307e-02,2.153001e-02,4.157142e-02,4.258126e-02,-7.948117e-03,4.769866e-02,6.383684e-02,3.923139e-02,-4.306484e-02,2.971388e-02,4.452496e-02,2.774148e-02,-1.965844e-02,2.662960e-02,2.849898e-02,1.095967e-02
std,9.999884e-01,1.001702e+00,1.008041e+00,1.004690e+00,1.003871e+00,9.909114e-01,9.970512e-01,9.986135e-01,9.982445e-01,9.997711e-01,1.000986e+00,9.983974e-01,1.000056e+00,1.001491e+00,1.001318e+00,1.000149e+00,9.979293e-01,9.987681e-01,9.962853e-01
min,-1.557116e+00,-1.834749e+00,-1.169063e+00,-1.449810e+00,-3.225297e+00,-3.119366e+00,-4.246944e+00,-5.153661e+00,-3.585160e+00,-3.425627e+00,-3.487261e+00,-5.278745e+00,-3.769542e+00,-3.306074e+00,-2.929187e+00,-5.154486e+00,-3.472672e+00,-3.678914e+00,-3.024405e+00
25%,-7.982832e-01,-5.439885e-01,-9.041336e-01,-1.041214e+00,-1.269445e+00,-7.914010e-01,-4.502157e-01,-4.446263e-01,-7.551433e-01,-6.484908e-01,-6.295125e-01,-7.297942e-01,-7.272984e-01,-8.990283e-01,-9.532807e-01,-6.380075e-01,-1.045526e+00,-3.143440e-01,-7.962322e-01
50%,-2.732248e-01,-4.617299e-02,3.949407e-01,1.102241e-01,3.739999e-01,-5.282060e-02,-2.770656e-01,-1.135471e-01,3.910827e-01,5.105792e-01,2.973343e-02,-8.255640e-02,8.095925e-02,1.326503e-01,5.696625e-02,5.870787e-03,8.394822e-02,2.799833e-02,7.706369e-02
75%,7.521172e-01,8.172447e-01,1.176564e+00,2.259460e-01,7.717648e-01,9.895137e-01,1.122608e+00,5.705792e-01,6.995134e-01,7.699772e-01,6.433953e-01,6.154810e-01,6.844449e-01,6.674007e-01,8.100300e-01,5.933324e-01,7.047071e-01,5.677800e-01,7.049202e-01
max,3.255552e+00,2.151099e+00,2.033639e+00,3.493843e+00,3.459681e+00,2.343346e+00,3.995469e+00,5.948771e+00,3.634544e+00,4.811091e+00,4.542479e+00,7.716150e+00,5.371619e+00,5.371699e+00,4.012976e+00,8.575139e+00,6.291730e+00,6.623074e+00,4.989589e+00


In [23]:
test_pre[conts].describe()

,angle,d_1_0,d_2_0,d_2_1,d_3_0,d_3_1,d_3_2,d_4_0,d_4_1,d_4_2,d_4_3,d_5_0,d_5_1,d_5_2,d_5_3,d_6_0,d_6_1,d_6_2,d_6_3
count,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06,2.505542e+06
mean,1.870482e-03,1.837534e-02,6.463940e-02,6.424154e-02,2.237896e-02,4.034468e-02,4.388591e-02,-9.822115e-03,4.936559e-02,6.435500e-02,3.918195e-02,-4.285982e-02,2.955620e-02,4.384346e-02,2.884435e-02,-1.869155e-02,2.716279e-02,2.872533e-02,1.191294e-02
std,1.002163e+00,1.004041e+00,1.001407e+00,1.007008e+00,1.002250e+00,9.944122e-01,1.005277e+00,1.000469e+00,1.000214e+00,1.000349e+00,1.004237e+00,1.000236e+00,1.000816e+00,1.001699e+00,1.000773e+00,1.001345e+00,9.992558e-01,1.001457e+00,9.994797e-01
min,-1.557116e+00,-1.834772e+00,-1.168822e+00,-1.449458e+00,-1.515590e+00,-1.447528e+00,-1.646960e+00,-5.153661e+00,-3.585160e+00,-3.425627e+00,-3.487261e+00,-5.278745e+00,-3.769542e+00,-3.306074e+00,-2.929187e+00,-5.154486e+00,-3.472672e+00,-3.678914e+00,-3.024405e+00
25%,-7.967794e-01,-5.435347e-01,-9.041300e-01,-1.041277e+00,-1.269415e+00,-7.957028e-01,-4.511842e-01,-4.486038e-01,-7.542096e-01,-6.503331e-01,-6.296535e-01,-7.297956e-01,-7.270318e-01,-8.988087e-01,-9.528455e-01,-6.371149e-01,-1.044045e+00,-3.144204e-01,-7.942262e-01
50%,-2.706272e-01,-4.731366e-02,3.936405e-01,1.102375e-01,3.778717e-01,-5.495989e-02,-2.765411e-01,-1.141000e-01,3.917809e-01,5.118628e-01,3.004983e-02,-8.274918e-02,8.060604e-02,1.327267e-01,6.035705e-02,6.195101e-03,8.428513e-02,2.931276e-02,7.783316e-02
75%,7.585490e-01,8.170904e-01,1.176181e+00,2.263631e-01,7.714604e-01,9.878154e-01,1.126413e+00,5.709616e-01,7.033592e-01,7.705477e-01,6.442838e-01,6.149552e-01,6.829773e-01,6.666362e-01,8.132419e-01,5.944929e-01,7.034352e-01,5.678880e-01,7.041502e-01
max,3.253247e+00,2.790596e+00,2.119213e+00,3.475595e+00,3.461015e+00,2.965536e+00,3.371320e+00,5.916121e+00,3.698970e+00,4.821081e+00,3.667066e+00,7.709538e+00,5.418262e+00,5.224132e+00,4.028304e+00,8.575206e+00,6.291515e+00,6.622846e+00,4.989412e+00


In [24]:
test_pre.isna().sum().sum(), train_pre.isna().sum().sum()

(0, 0)

In [25]:
train_pre.head()

,id,molecule_name,atom_index_0,atom_index_1,type_i,type_a,type,atom_2,atom_3,atom_4,atom_5,atom_6,angle,d_1_0,d_2_0,d_2_1,d_3_0,d_3_1,d_3_2,d_4_0,d_4_1,d_4_2,d_4_3,d_5_0,d_5_1,d_5_2,d_5_3,d_6_0,d_6_1,d_6_2,d_6_3,scalar_coupling_constant
0,0,dsgdb9nsd_000001,2,1,0,0,0,1,1,1,0,0,2.570007,-1.712379,0.454218,-1.066589,-0.045399,-1.218631,0.376567,-1.099632,-1.553749,-0.047694,-0.615493,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405,84.807602
1,1,dsgdb9nsd_000001,2,3,1,1,3,2,1,1,0,0,1.755858,-0.769607,-0.906534,-1.066589,-0.045383,-0.015466,-1.415656,-1.099652,-0.267853,-1.357082,-0.615493,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405,-11.257000
2,2,dsgdb9nsd_000001,2,4,1,1,3,2,1,1,0,0,2.922918,-0.769569,-0.906534,-1.066604,-0.045449,-0.015450,-1.415646,-1.099632,-0.267871,-1.357080,-0.615492,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405,-11.254800
3,3,dsgdb9nsd_000001,2,5,1,1,3,2,1,1,0,0,1.977800,-0.769557,-0.906534,-1.066601,-0.045449,-0.015466,-1.415646,-1.099652,-0.267871,-1.357082,-0.615478,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405,-11.254300
4,4,dsgdb9nsd_000001,3,1,0,0,0,1,1,1,0,0,-0.612658,-1.712381,0.454218,-1.066586,-0.045398,-1.218629,0.376565,-1.099630,-1.553752,-0.047696,-0.615493,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405,84.807404


In [26]:
test_pre.head()

,id,molecule_name,atom_index_0,atom_index_1,type_i,type_a,type,atom_2,atom_3,atom_4,atom_5,atom_6,angle,d_1_0,d_2_0,d_2_1,d_3_0,d_3_1,d_3_2,d_4_0,d_4_1,d_4_2,d_4_3,d_5_0,d_5_1,d_5_2,d_5_3,d_6_0,d_6_1,d_6_2,d_6_3
0,4658147,dsgdb9nsd_000004,3,1,1,0,2,2,1,0,0,0,0.846656,-0.117521,-0.965310,-0.758715,2.701123,-1.270586,1.616017,-5.153661,-3.58516,-3.425627,-3.487261,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405
1,4658148,dsgdb9nsd_000004,3,2,0,0,0,2,1,0,0,0,-0.808065,-1.753100,1.395408,-0.758715,2.701123,0.816633,-1.493050,-5.153661,-3.58516,-3.425627,-3.487261,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405
2,4658149,dsgdb9nsd_000004,3,4,2,1,6,2,2,0,0,0,1.595707,1.331213,1.395408,-1.152383,-1.331249,0.816633,-1.137877,-5.153661,-3.58516,-3.425627,-3.487261,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405
3,4658150,dsgdb9nsd_000004,4,1,0,0,0,2,1,0,0,0,-0.808065,-1.753100,1.395408,-0.758715,2.701123,0.816633,-1.493050,-5.153661,-3.58516,-3.425627,-3.487261,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405
4,4658151,dsgdb9nsd_000004,4,2,1,0,2,2,1,0,0,0,0.846656,-0.117521,-0.965310,-0.758715,2.701123,-1.270586,1.616017,-5.153661,-3.58516,-3.425627,-3.487261,-5.278745,-3.769542,-3.306074,-2.929187,-5.154486,-3.472672,-3.678914,-3.024405


In [27]:
with open('train_test_pre.pkl', 'wb') as f:
    pickle.dump([train_pre, test_pre, column_names], f)

In [28]:
train_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4658147 entries, 0 to 4658146
Data columns (total 32 columns):
id                          int64
molecule_name               object
atom_index_0                int8
atom_index_1                int8
type_i                      int8
type_a                      int8
type                        int8
atom_2                      int8
atom_3                      int8
atom_4                      int8
atom_5                      int8
atom_6                      int8
angle                       float32
d_1_0                       float32
d_2_0                       float32
d_2_1                       float32
d_3_0                       float32
d_3_1                       float32
d_3_2                       float32
d_4_0                       float32
d_4_1                       float32
d_4_2                       float32
d_4_3                       float32
d_5_0                       float32
d_5_1                       float32
d_5_2               

In [29]:
train_pre.memory_usage(deep=True).sum()/1024/1024

759.6428079605103